In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, get_anni_op_mpo
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import unittest

In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [3]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

e_ground = -84.9226298311996

[1, 16, 58, 96, 96, 58, 16, 1]


In [4]:
# psi = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])

# en_min, psi_min = ptn.calculate_ground_state_local_twosite(mpo_ref, psi, 10, 25, 0)
# e0 = en_min[-1]
# print(e0 - e_ground)
# print(psi_min.bond_dims)

In [5]:
# filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(psi_min, file)

In [6]:
filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground.pkl"
with open(filename, 'rb') as file:
    water_ground = pickle.load(file)

print(ptn.operation.operator_average(water_ground, mpo_ref) - e_ground)

(-9.279688129026908e-12+0j)


In [58]:
anni_3_up = get_anni_op_mpo(3, 0, 7)
initial = ptn.operation.apply_operator(anni_3_up, copy.deepcopy(water_ground))

test: (attention, here the index for ED is inversed!)

In [59]:
norm((ptn.fermi_sim.annihil_op(14, 2**(7)) @ (copy.deepcopy(water_ground)).as_vector()) + initial.as_vector())

0.0

In [ ]:
# initial.orthonormalize('right')

# filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(initial, file)